In [116]:
import ipywidgets as widgets
import itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import yfinance as yf

In [117]:
# Отрисовка свечного графика
def candles(ax, data, interval, alpha=1):
    coeff = {"1m":1/24/60, "2m":1/24/30, "5m":1/24/12, "15m":1/24/4, "30m":1/24/2,
        "60m":1/24, "90m":1/16, "1h":1/24, "1d":1, "5d":5, "1wk":7, "1mo":30,
         "3mo":90}
    up = data[data.Close >= data.Open]
    down = data[data.Close < data.Open]
    col_up = 'green'
    col_down = 'red'
    col_shadow_up = 'black'
    col_shadow_down = 'black'
    width_body = .8 * coeff[interval]
    width_shadow = .2 * coeff[interval]
    ax.bar(up.index, up.Close-up.Open, width_body, bottom=up.Open, color=col_up, alpha=alpha)
    ax.bar(up.index, up.High-up.Close, width_shadow, bottom=up.Close, color=col_shadow_up, alpha=alpha)
    ax.bar(up.index, up.Low-up.Open, width_shadow, bottom=up.Open, color=col_shadow_up, alpha=alpha)
    ax.bar(down.index, down.Close-down.Open, width_body, bottom=down.Open, color=col_down, alpha=alpha)
    ax.bar(down.index, down.High-down.Open, width_shadow, bottom=down.Open, color=col_shadow_down, alpha=alpha)
    ax.bar(down.index, down.Low-down.Close, width_shadow, bottom=down.Close, color=col_shadow_down, alpha=alpha)

In [118]:
# Получение данных
def get_data(ticker, period, interval):
    return yf.download(ticker, period=period, interval=interval)

In [119]:
# Нахождение экстремумов
def get_points(data, n):
    resistance_points = scipy.signal.argrelextrema(data["High"].to_numpy(), np.greater, axis=0, order=n)
    support_points = scipy.signal.argrelextrema(data["Low"].to_numpy(), np.less, axis=0, order=n)
    return resistance_points, support_points

In [127]:
# Определение линий тренда
def get_levels(data, resistance_points, support_points, pivots_number):
    resistance_points = list(zip(resistance_points[0], data.iloc[resistance_points]["High"], ["Resistance" for _ in range(len(resistance_points[0]))]))
    support_points = list(zip(support_points[0], data.iloc[support_points]["Low"], ["Support" for _ in range(len(support_points[0]))]))
    
    points = sorted(resistance_points + support_points, key=lambda x: x[0])
    
    result = []
    
    UP = 1
    DOWN = 0
    
    last_resistance = None
    last_support = None
    streak = 0
    direction = None
    for i in range(len(points)):
        resistance = (points[i][2] == "Resistance")
        if resistance:
            if last_resistance is None:
                last_resistance = points[i][1]
                streak += 1
                continue
            else:
                last = last_resistance 
        else:
            if last_support is None:
                last_support = points[i][1]
                streak += 1
                continue
            else:
                last = last_support    
          
        if direction is None:
            if points[i][1] >= last:
                direction = UP
            else:
                direction = DOWN
            streak += 1
            continue
            
        if (points[i][1] >= last) == direction:
            if resistance:
                last_resistance = points[i][1]
            else:
                last_support = points[i][1]
            streak += 1
            continue
        else:
            if streak >= pivots_number:
                result.append(points[i-streak:i])
            last_resistance = None
            last_support = None
            streak = 0
            direction = None
    if streak >= pivots_number:
        result.append(points[i-streak:i])
        
    temp = result
    result = []
    for res in temp:
        x_r, y_r, x_s, y_s = [], [], [], []
        for row in res:
            if row[2] == "Resistance":
                x_r.append(row[0])
                y_r.append(row[1])
            else:
                x_s.append(row[0])
                y_s.append(row[1])   
        w_r = np.polyfit(x_r, y_r, 1)
        w_s = np.polyfit(x_s, y_s, 1)
        start = res[0][0]
        end = res[-1][0]
        result.append({"Resistance": [data.iloc[[start, end]].index, 
                                      [start * w_r[0] + w_r[1], end * w_r[0] + w_r[1]]], 
                       "Support": [data.iloc[[start, end]].index, 
                                   [start * w_s[0] + w_s[1], end * w_s[0] + w_s[1]]]})
    return result

In [130]:
def res(ticker="BTC-USD", period="1y", interval="1d", divider=80, pivots_number=8):
    data = get_data(ticker, period, interval)
    
    n = round(data.shape[0]/divider)
    if not n:
        n = 1
        
    resistance_points, support_points = get_points(data, n)
    levels = get_levels(data, resistance_points, support_points, pivots_number)
    
    fig = plt.figure(figsize=[12.8, 9.6])

    ax1 = fig.add_subplot(221)
    candles(ax1, data, interval)
    ax1.set_title("Исходные данные")

    ax2 = fig.add_subplot(222)
    candles(ax2, data, interval, 0.5)
    ax2.scatter(data.iloc[resistance_points].index, data.iloc[resistance_points]["High"], color="green")
    ax2.scatter(data.iloc[support_points].index, data.iloc[support_points]["Low"], color="red")
    ax2.set_title("Экстремумы")

    ax3 = fig.add_subplot(223)
    candles(ax3, data, interval, 0.5)
    ax3.scatter(data.iloc[resistance_points].index, data.iloc[resistance_points]["High"], color="green")
    ax3.scatter(data.iloc[support_points].index, data.iloc[support_points]["Low"], color="red")
    for lvl in levels:
        ax3.plot(lvl["Resistance"][0], lvl["Resistance"][1], lw=2, c="blue")
        ax3.plot(lvl["Support"][0], lvl["Support"][1], lw=2, c="blue")
    ax3.set_title("Экстремумы и уровни")

    ax4 = fig.add_subplot(224)
    candles(ax4, data, interval)
    for lvl in levels:
        ax4.plot(lvl["Resistance"][0], lvl["Resistance"][1], lw=2, c="blue")
        ax4.plot(lvl["Support"][0], lvl["Support"][1], lw=2, c="blue")
    ax4.set_title("Результат")

    fig.suptitle(f"{ticker}, n = {n}")
    fig.subplots_adjust(top=0.94, hspace=0.3)
    for ax in fig.axes:
        plt.setp(ax.get_xticklabels(), rotation=30)
    plt.show()
    
ticker_list = ["BTC-USD", "ETH-USD", "USDT-USD", "BNB-USD", "USDC-USD", 
               "XRP-USD", "ADA-USD", "DOGE-USD", "STETH-USD", "HEX-USD", 
               "MATIC-USD", "SOL-USD", "DOT-USD", "LTC-USD", "WTRX-USD"]
period_list = ["1h", "1d", "1wk", "1mo", "3mo", "6mo", "1y", "2y", "3y", "5y"]
interval_list = ["1m", "2m", "5m", "15m", "30m", "60m", "90m", "1h", "1d", "5d", "1wk", "1mo", "3mo"]

interactive_plot = widgets.interactive(res, {'manual': True}, ticker=ticker_list, period=period_list, interval=interval_list, 
                                       divider=(30, 200, 10), pivots_number=(5, 20, 1))
output = interactive_plot.children[-1]
output.layout.height = '1000px'
interactive_plot

interactive(children=(Dropdown(description='ticker', options=('BTC-USD', 'ETH-USD', 'USDT-USD', 'BNB-USD', 'US…